In [1]:
from google.cloud import bigquery
import gc
import pandas as pd
import numpy as np
import datetime
import os
import json
import glob
from haversine import haversine
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

zip_centers=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))
client = bigquery.Client()

print(datetime.datetime.now())
print(os.getcwd())

2020-11-03 14:17:43.621071
/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections


In [2]:
dict_date_range={"R1H":['2019-10-01','2020-09-30'],
                 "R2M":['2017-04-01','2019-09-30'],
                 "R3L":[None,'2017-03-31']}

In [3]:
# Part 1 40 stores
'''
df_selected_stores=pd.read_excel("./GY_40_selected_green_stores_JL_2020-10-30.xlsx",dtype=str)
df_selected_stores['store_id']=df_selected_stores['index'].apply(lambda x: int(x.split("|")[-1].replace(" ","")))

print(df_selected_stores.shape)
list_test_store_str_id_part_1=[str(x).zfill(4) for x in df_selected_stores['store_id'].unique()]
print(len(list_test_store_str_id_part_1))
df_selected_stores.head(3)
'''

'\ndf_selected_stores=pd.read_excel("./GY_40_selected_green_stores_JL_2020-10-30.xlsx",dtype=str)\ndf_selected_stores[\'store_id\']=df_selected_stores[\'index\'].apply(lambda x: int(x.split("|")[-1].replace(" ","")))\n\nprint(df_selected_stores.shape)\nlist_test_store_str_id_part_1=[str(x).zfill(4) for x in df_selected_stores[\'store_id\'].unique()]\nprint(len(list_test_store_str_id_part_1))\ndf_selected_stores.head(3)\n'

In [4]:
# Part 2 single store TAs 
'''
df_store_list_sigle_TA=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/TA_creation/output_2020-10-23/GY_final_TA_updated_JL_2020-10-23.xlsx",
                           dtype=str,sheet_name="view_by_TA")
df_store_list_sigle_TA['store_count']=df_store_list_sigle_TA['store_count'].astype(int)
df_store_list_sigle_TA=df_store_list_sigle_TA[df_store_list_sigle_TA['store_count']==1]
df_store_list_sigle_TA['count_Coor_store_in_P']=df_store_list_sigle_TA['count_Coor_store_in_P'].astype(int)
df_store_list_sigle_TA['count_Noncoor_store_in_P']=df_store_list_sigle_TA['count_Noncoor_store_in_P'].astype(int)
df_store_list_sigle_TA=df_store_list_sigle_TA[df_store_list_sigle_TA['count_Coor_store_in_P']>df_store_list_sigle_TA['count_Noncoor_store_in_P']]
df_store_list_sigle_TA.reset_index(inplace=True,drop=True)
list_store_str_id_part_2=[]
for i, row in df_store_list_sigle_TA.iterrows():
    list_store_str_id_part_2.extend(eval(row['store_list']))
list_test_store_str_id_part_2=[str(int(x)).zfill(4) for x in list_store_str_id_part_2]
print(len(list_test_store_str_id_part_2))
list_test_store_str_id_part_2
'''

'\ndf_store_list_sigle_TA=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/TA_creation/output_2020-10-23/GY_final_TA_updated_JL_2020-10-23.xlsx",\n                           dtype=str,sheet_name="view_by_TA")\ndf_store_list_sigle_TA[\'store_count\']=df_store_list_sigle_TA[\'store_count\'].astype(int)\ndf_store_list_sigle_TA=df_store_list_sigle_TA[df_store_list_sigle_TA[\'store_count\']==1]\ndf_store_list_sigle_TA[\'count_Coor_store_in_P\']=df_store_list_sigle_TA[\'count_Coor_store_in_P\'].astype(int)\ndf_store_list_sigle_TA[\'count_Noncoor_store_in_P\']=df_store_list_sigle_TA[\'count_Noncoor_store_in_P\'].astype(int)\ndf_store_list_sigle_TA=df_store_list_sigle_TA[df_store_list_sigle_TA[\'count_Coor_store_in_P\']>df_store_list_sigle_TA[\'count_Noncoor_store_in_P\']]\ndf_store_list_sigle_TA.reset_index(inplace=True,drop=True)\nlist_store_str_id_part_2=[]\nfor i, row in df_store_list_sigle_TA.iterrows():\n    list_store_str_id_part_2.extend(eval(row[\'store_list\']))

In [5]:
df_31_green=pd.read_excel("./GY_green_store_matching_pair_BenJL_20201103.xlsx",sheet_name="Sheet1",dtype=str)
df_9_single=pd.read_excel("./GY_twelve_matching_pair_BenJL_20201103.xlsx",sheet_name="Sheet1",dtype=str)
df_31_green=df_31_green[df_31_green['final_selection']=="1"]
df_9_single=df_9_single[df_9_single['final_selection']=="1"]

# 1
list_test_store_str_id_part_1=[str(int(x)).zfill(4) for x in df_31_green['green_store_id'].tolist()]
list_control_store_str_id_part_1=[str(int(x)).zfill(4) for x in df_31_green['matching_store_id'].tolist()]
# 2
list_test_store_str_id_part_2=[str(int(x)).zfill(4) for x in df_9_single['twelve_store_id'].tolist()]
list_control_store_str_id_part_2=[str(int(x)).zfill(4) for x in df_9_single['matching_store_id'].tolist()]

print(len(list_test_store_str_id_part_1),len(list_control_store_str_id_part_1))
print(len(list_test_store_str_id_part_2),len(list_control_store_str_id_part_2))

print(len(set(list_test_store_str_id_part_1+list_control_store_str_id_part_1+list_test_store_str_id_part_2+list_control_store_str_id_part_2)))

31 31
9 9
80


In [6]:
df_store_view=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/TA_creation/output_2020-10-23/GY_final_TA_updated_JL_2020-10-23.xlsx",
                           dtype=str,sheet_name="view_by_store")
df_store_view=df_store_view[['TA_num','store_id','trans_P_zips_40_within_TA','trans_S_zips_40_within_TA']]
df_store_view['store_id']=df_store_view['store_id'].astype(int).astype(str).str.zfill(4)


In [7]:
'''
df_qc=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/TA_creation/output_2020-10-23/GY_final_TA_updated_JL_2020-10-23.xlsx",
                           dtype=str,sheet_name="view_for_Tableau")
df_qc['store_id']=df_qc['store_id'].astype(int)
df_qc=df_qc[df_qc['store_id'].isin(df_selected_stores['store_id'].tolist())]
print(df_qc.shape)
print(df_qc['store_id'].nunique())
print(df_qc['TA_num'].nunique())
print(df_qc['zip_cd'].nunique())
'''

'\ndf_qc=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/TA_creation/output_2020-10-23/GY_final_TA_updated_JL_2020-10-23.xlsx",\n                           dtype=str,sheet_name="view_for_Tableau")\ndf_qc[\'store_id\']=df_qc[\'store_id\'].astype(int)\ndf_qc=df_qc[df_qc[\'store_id\'].isin(df_selected_stores[\'store_id\'].tolist())]\nprint(df_qc.shape)\nprint(df_qc[\'store_id\'].nunique())\nprint(df_qc[\'TA_num\'].nunique())\nprint(df_qc[\'zip_cd\'].nunique())\n'

In [8]:
class res_GY_audience():
    def __init__(self,store_list,df_TA_store_view,BQ_client,dict_date_range,TIRE_only,test_or_control,audience_name=None):
        if audience_name:
            self.audience_name=audience_name
        else:
            if TIRE_only:
                self.audience_name="%s_TireOnly_%i_stores"%(test_or_control,len(store_list))
            else:
                self.audience_name="%s_TireAndService_%i_stores"%(test_or_control,len(store_list))
            
        self.store_list=store_list
        self.df_selected_store=df_TA_store_view[df_TA_store_view['store_id'].isin(store_list)]
        self.df_selected_store.reset_index(inplace=True,drop=True)
        self.TIRE_only=TIRE_only
        
        self.client=BQ_client
        self.recency_date_range=dict_date_range
        
        self.output_seleted_store_zip_long=pd.DataFrame()
        self.list_P=[]
        self.list_S=[]   
        self.df_ids_ECOM_all=pd.DataFrame()
        self.df_ids_Store_all=pd.DataFrame()
        
        self.output_aud_path="./output_audience_%s/"%str(datetime.datetime.now().date())
        self.output_zip_path="./output_zip_%s/"%str(datetime.datetime.now().date())
        try:
            os.stat(self.output_aud_path)
        except:
            os.mkdir(self.output_aud_path)
        
        try:
            os.stat(self.output_zip_path)
        except:
            os.mkdir(self.output_zip_path)
        
        
    def get_zip_df_and_audience_from_store(self):
        df_output_PS_by_store_long=pd.DataFrame()
        for ind, row in self.df_selected_store.iterrows():
            TA_num=row['TA_num']
            store_id=row['store_id']
            list_P=eval(row['trans_P_zips_40_within_TA'])
            list_S=eval(row['trans_S_zips_40_within_TA'])
            df_P=pd.DataFrame({"TA_num":[TA_num]*len(list_P),
                               "store_id":[store_id]*len(list_P),
                               "zip_label":["P"]*len(list_P),
                               "zip_cd":list_P},index=[store_id]*len(list_P))
            df_S=pd.DataFrame({"TA_num":[TA_num]*len(list_S),
                               "store_id":[store_id]*len(list_S),
                               "zip_label":["S"]*len(list_S),
                               "zip_cd":list_S},index=[store_id]*len(list_S))
            df_output_PS_by_store_long=df_output_PS_by_store_long.append(df_P).append(df_S)
        print(df_output_PS_by_store_long.shape)
        print(df_output_PS_by_store_long['store_id'].nunique())
        print(df_output_PS_by_store_long['TA_num'].nunique())
        print(df_output_PS_by_store_long['zip_cd'].nunique())

        list_P=df_output_PS_by_store_long[df_output_PS_by_store_long['zip_label']=="P"]['zip_cd'].unique().tolist()
        list_S=df_output_PS_by_store_long[df_output_PS_by_store_long['zip_label']=="S"]['zip_cd'].unique().tolist()
        # list_S=[x for x in list_S if x not in list_P]
        print("len(list_P): ",len(list_P))
        print("len(list_S): ",len(list_S))
        
        self.output_seleted_store_zip_long=df_output_PS_by_store_long
        self.list_P=list_P
        self.list_S=list_S
        
    def pull_all_ids_with_latest_trans_date(self):
        str_store_list=str(self.store_list).replace("[","(").replace("]",")")
        if self.TIRE_only==True:
            query_ecom="""
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            """
            query_store="""
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in %s and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            """%str_store_list
        else:
            query_ecom="""
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null
            group by indiv_key
            """
            query_store="""
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in %s and indiv_key is not null
            group by indiv_key
            """%str_store_list
            
        
        print("query_ecom: \n",query_ecom)
        print("query_store: \n",query_store)
        
        self.df_ids_ECOM_all=self.client.query(query_ecom).result().to_dataframe()
        self.df_ids_ECOM_all=self.df_ids_ECOM_all.sort_values(['indiv_key','last_trans_date'],ascending=[True,False]).drop_duplicates('indiv_key')
        self.df_ids_ECOM_all['last_trans_date']=self.df_ids_ECOM_all['last_trans_date'].astype(str).str[:10]
        
        self.df_ids_Store_all=self.client.query(query_store).result().to_dataframe()
        self.df_ids_Store_all=self.df_ids_Store_all.sort_values(['indiv_key','last_trans_date'],ascending=[True,False]).drop_duplicates('indiv_key')
        self.df_ids_Store_all['last_trans_date']=self.df_ids_Store_all['last_trans_date'].astype(str).str[:10]
        
    def pull_all_id_with_zip(self):
        # 1 indiv_key match multiple cr_indiv_key which result in multiple postal_cd
        # Here kept there last records
        query_crm="""
        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        """
        
# no needed since we can use the cr_addr_key in the indiv_summary table    
        '''
                with t_mapping as (SELECT                                                                                                             
    indiv_key, cr_addr_key                                                                                                               
  FROM (                                                                                                             
    SELECT                                                                                                           
      indiv_key,                                                                                                     
      cr_addr_key,                                                                                                   
      ia.file_id,                                                                                                    
      ia.record_id,                                                                                                  
      ia.create_process_log_id,                                                                                      
      ia.update_process_log_id,                                                                                      
      ROW_NUMBER() OVER (PARTITION BY indiv_key ORDER BY ia.file_id DESC, ia.record_id DESC) AS ia_rnk  
    FROM                                                                                                             
      `merkle-gdyr-prod.mdb.indiv_addr` ia                                                                            
    JOIN                                                                                                             
      `merkle-gdyr-prod.mdb.indiv` i                                                                                  
    ON                                                                                                               
      ia.cr_indiv_key = i.cr_indiv_key ) univ                                                                        
  WHERE                                                                                                              
    ia_rnk = 1)
        
        '''
        print(query_crm)
        self.df_all_ids=self.client.query(query_crm).result().to_dataframe()
    def generate_audience_df(self):
        df_Ecom=pd.merge(self.df_ids_ECOM_all,self.df_all_ids,on="indiv_key",how="left")
        self.df_Ecom=df_Ecom[df_Ecom['postal_cd'].isin(self.list_P+self.list_S)]
        '''
        self.df_1_Ecom_P_R1H=df_Ecom[(df_Ecom['postal_cd'].isin(self.list_P)) & (df_Ecom['last_trans_date']>=self.recency_date_range['R1H'][0]) & (df_Ecom['last_trans_date']<=self.recency_date_range['R1H'][1])]
        self.df_2_Ecom_P_R2M=df_Ecom[(df_Ecom['postal_cd'].isin(self.list_P)) & (df_Ecom['last_trans_date']>=self.recency_date_range['R2M'][0]) & (df_Ecom['last_trans_date']<=self.recency_date_range['R2M'][1])]
        self.df_3_Ecom_P_R3L=df_Ecom[(df_Ecom['postal_cd'].isin(self.list_P)) & (df_Ecom['last_trans_date']<=self.recency_date_range['R3L'][1])]
        
        self.df_4_Ecom_S_R1H=df_Ecom[(df_Ecom['postal_cd'].isin(self.list_S)) & (df_Ecom['last_trans_date']>=self.recency_date_range['R1H'][0]) & (df_Ecom['last_trans_date']<=self.recency_date_range['R1H'][1])]
        self.df_5_Ecom_S_R2M=df_Ecom[(df_Ecom['postal_cd'].isin(self.list_S)) & (df_Ecom['last_trans_date']>=self.recency_date_range['R2M'][0]) & (df_Ecom['last_trans_date']<=self.recency_date_range['R2M'][1])]
        self.df_6_Ecom_S_R3L=df_Ecom[(df_Ecom['postal_cd'].isin(self.list_S)) & (df_Ecom['last_trans_date']<=self.recency_date_range['R3L'][1])]
        '''
        #
        df_Store=pd.merge(self.df_ids_Store_all,self.df_all_ids,on="indiv_key",how="left")
        self.df_7_Store_P_R1H=df_Store[(df_Store['postal_cd'].isin(self.list_P)) & (df_Store['last_trans_date']>=self.recency_date_range['R1H'][0]) & (df_Store['last_trans_date']<=self.recency_date_range['R1H'][1])]
        self.df_8_Store_P_R2M=df_Store[(df_Store['postal_cd'].isin(self.list_P)) & (df_Store['last_trans_date']>=self.recency_date_range['R2M'][0]) & (df_Store['last_trans_date']<=self.recency_date_range['R2M'][1])]
        self.df_9_Store_P_R3L=df_Store[(df_Store['postal_cd'].isin(self.list_P)) & (df_Store['last_trans_date']<=self.recency_date_range['R3L'][1])]
        
        self.df_10_Store_S_R1H=df_Store[(df_Store['postal_cd'].isin(self.list_S)) & (df_Store['last_trans_date']>=self.recency_date_range['R1H'][0]) & (df_Store['last_trans_date']<=self.recency_date_range['R1H'][1])]
        self.df_11_Store_S_R2M=df_Store[(df_Store['postal_cd'].isin(self.list_S)) & (df_Store['last_trans_date']>=self.recency_date_range['R2M'][0]) & (df_Store['last_trans_date']<=self.recency_date_range['R2M'][1])]
        self.df_12_Store_S_R3L=df_Store[(df_Store['postal_cd'].isin(self.list_S)) & (df_Store['last_trans_date']<=self.recency_date_range['R3L'][1])]

    def write_audience_files(self):
        self.output_seleted_store_zip_long.to_csv(self.output_zip_path+"GY_selected_zips_%s_JL_%s.csv"%(self.audience_name,str(datetime.datetime.now().date())),index=False)
        '''
        self.df_1_Ecom_P_R1H.to_csv(self.output_aud_path+"segment_%s_Ecom_P_R1H.csv"%self.audience_name,index=False)
        self.df_2_Ecom_P_R2M.to_csv(self.output_aud_path+"segment_%s_Ecom_P_R2M.csv"%self.audience_name,index=False)
        self.df_3_Ecom_P_R3L.to_csv(self.output_aud_path+"segment_%s_Ecom_P_R3L.csv"%self.audience_name,index=False)
        self.df_4_Ecom_S_R1H.to_csv(self.output_aud_path+"segment_%s_Ecom_S_R1H.csv"%self.audience_name,index=False)
        self.df_5_Ecom_S_R2M.to_csv(self.output_aud_path+"segment_%s_Ecom_S_R2M.csv"%self.audience_name,index=False)
        self.df_6_Ecom_S_R3L.to_csv(self.output_aud_path+"segment_%s_Ecom_S_R3L.csv"%self.audience_name,index=False)
        '''
        self.df_Ecom.to_csv(self.output_aud_path+"segment_%s_Ecommerce_All_All.csv"%self.audience_name,index=False)
        self.df_7_Store_P_R1H.to_csv(self.output_aud_path+"segment_%s_Store_P_R1H.csv"%self.audience_name,index=False)
        self.df_8_Store_P_R2M.to_csv(self.output_aud_path+"segment_%s_Store_P_R2M.csv"%self.audience_name,index=False)
        self.df_9_Store_P_R3L.to_csv(self.output_aud_path+"segment_%s_Store_P_R3L.csv"%self.audience_name,index=False)
        self.df_10_Store_S_R1H.to_csv(self.output_aud_path+"segment_%s_Store_S_R1H.csv"%self.audience_name,index=False)
        self.df_11_Store_S_R2M.to_csv(self.output_aud_path+"segment_%s_Store_S_R2M.csv"%self.audience_name,index=False)
        self.df_12_Store_S_R3L.to_csv(self.output_aud_path+"segment_%s_Store_S_R3L.csv"%self.audience_name,index=False)



In [9]:
# Test
# 40 stores
# Tire only

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_test_store_str_id_part_1,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=True,
                         test_or_control="Test",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(2660, 4)
31
16
1544
len(list_P):  73
len(list_S):  1529
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('2728', '2178', '2842', '4128', '6227', '0431', '2677', '2750', '4246', '6658', '2724', '2733', '6626', '2248', '4739', '2621', '4328', '8963', '2844', '4738', '2766', '2759', '1224', '2180', '4722', '6258', '4730', '2251', '2777', '6257', '2249') and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            


/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "



        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [10]:
# Control
# 40 stores
# Tire only

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_control_store_str_id_part_1,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=True,
                         test_or_control="Control",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(5455, 4)
31
25
4388
len(list_P):  126
len(list_S):  4312
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('6661', '4248', '8259', '2523', '0231', '2325', '2654', '0845', '4733', '8836', '6552', '8152', '4330', '2476', '4230', '6641', '2647', '8650', '2424', '2381', '0933', '0544', '1271', '4724', '2755', '6663', '0848', '8766', '0461', '2484', '6644') and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [11]:
# Test
# 40 stores
# Tire&Service

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_test_store_str_id_part_1,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=False,
                         test_or_control="Test",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(2660, 4)
31
16
1544
len(list_P):  73
len(list_S):  1529
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('2728', '2178', '2842', '4128', '6227', '0431', '2677', '2750', '4246', '6658', '2724', '2733', '6626', '2248', '4739', '2621', '4328', '8963', '2844', '4738', '2766', '2759', '1224', '2180', '4722', '6258', '4730', '2251', '2777', '6257', '2249') and indiv_key is not null
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [12]:
# Control
# 40 stores
# Tire&Service

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_control_store_str_id_part_1,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=False,
                         test_or_control="Control",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(5455, 4)
31
25
4388
len(list_P):  126
len(list_S):  4312
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('6661', '4248', '8259', '2523', '0231', '2325', '2654', '0845', '4733', '8836', '6552', '8152', '4330', '2476', '4230', '6641', '2647', '8650', '2424', '2381', '0933', '0544', '1271', '4724', '2755', '6663', '0848', '8766', '0461', '2484', '6644') and indiv_key is not null
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [13]:
# Test
# 14 stores
# TireOnly

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_test_store_str_id_part_2,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=True,
                         test_or_control="Test",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(575, 4)
9
9
575
len(list_P):  27
len(list_S):  548
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('6568', '6329', '2928', '6325', '4736', '2761', '2680', '8426', '9124') and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [14]:
# Control
# 14 stores
# TireOnly

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_control_store_str_id_part_2,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=True,
                         test_or_control="Control",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(928, 4)
9
9
926
len(list_P):  25
len(list_S):  901
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('8129', '2485', '2521', '2121', '0466', '6272', '6769', '1139', '2450') and indiv_key is not null and product_type_cd='TIRE'
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [15]:
# Test
# 14 stores
# Tire&Service

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_test_store_str_id_part_2,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=False,
                         test_or_control="Test",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(575, 4)
9
9
575
len(list_P):  27
len(list_S):  548
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('6568', '6329', '2928', '6325', '4736', '2761', '2680', '8426', '9124') and indiv_key is not null
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        


In [16]:
# Control
# 14 stores
# Tire&Service

res_GY_audience.__init__(self=res_GY_audience,
                         store_list=list_control_store_str_id_part_2,
                         df_TA_store_view=df_store_view,
                         BQ_client=client,
                         dict_date_range=dict_date_range,
                         TIRE_only=False,
                         test_or_control="Control",
                         audience_name=None
                        )

res_GY_audience.get_zip_df_and_audience_from_store(self=res_GY_audience)
res_GY_audience.pull_all_ids_with_latest_trans_date(self=res_GY_audience)
res_GY_audience.pull_all_id_with_zip(self=res_GY_audience)
res_GY_audience.generate_audience_df(self=res_GY_audience)
res_GY_audience.write_audience_files(self=res_GY_audience)


(928, 4)
9
9
926
len(list_P):  25
len(list_S):  901
query_ecom: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id='ECOM' and indiv_key is not null
            group by indiv_key
            
query_store: 
 
            select indiv_key, max(invoice_date) as last_trans_date from mdb.trn_detail
            where store_id in ('8129', '2485', '2521', '2121', '0466', '6272', '6769', '1139', '2450') and indiv_key is not null
            group by indiv_key
            

        select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd from mdb.indiv_summary as t1
        join mdb.addr as t2 on
        t1.cr_addr_key=t2.cr_addr_key
        
